In [51]:
# Based on https://github.com/MarkPKCollier/NeuralTuringMachine
# Wire up the required libraries
import sys
# Add folder with implementation to the system path
sys.path.insert(0, 'NeuralTuringMachine')

# Import the rest of the libraries
import numpy as np
import pandas as pd
import re
from ntm import NTMCell

In [30]:
# Generate the data

# Set the seed for reproducibility
np.random.seed(42)

# Split the data into training and test sets
from sklearn.model_selection import train_test_split

# Generate array of 1 and 0 with 10000 rows and 100 columns
X = np.random.randint(2, size=(10000, 100))

# Define regular expression
regex = "000000" # 6 zeros - pattern encountered approximately ~1/2 of the time
pattern = re.compile(regex)

# Apply regular expression to each row of X
y = np.array([ [1.0, 0.0] if pattern.search(''.join(map(str, row))) else [0.0, 1.0] for row in X])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


5397


In [54]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from utils import expand, learned_init


model = Sequential()

batch_size = 100
number_of_controller_layers = 1
number_of_controller_units = 128
max_series_length = 100
max_grad_norm = 50

# Compatibility https://www.tensorflow.org/api_docs/python/tf/compat/v1/reset_default_graph
tf.compat.v1.reset_default_graph()

cell = NTMCell(number_of_controller_layers,             # Number of recurrent controller layers
               number_of_controller_units,              # Number of units in the controller layer
               128,                                     # Number of memory locations (N)
               20,                                      # The vector size at each location (M)
               1,                                       # Number of read heads (R)
               1,                                       # Number of write heads (W)
               output_dim=1,                            # The output dimension in bits
               clip_value=20,                           # Maximum absolute value of controller and outputs.
               shift_range=1,                           # The range of the shift operation
               addressing_mode='content_and_location',  # Ways to access memory - using hybrid of content and location based addressing
               reuse=False,                             # Whether to reuse the weights of a previous layer by the same name.
               init_mode='constant')                    # Initialization mode for the weights of the controller, recommended setting.

# ---------------

initial_state = tuple(tf.contrib.rnn.LSTMStateTuple(
    c=expand(tf.tanh(learned_init(number_of_controller_units)), dim=0, N=batch_size),
    h=expand(tf.tanh(learned_init(number_of_controller_units)), dim=0, N=batch_size))
                      for _ in range(number_of_controller_layers))



inputs_placeholder = tf.placeholder(tf.float32, shape=(batch_size, None, max_series_length))
outputs_placeholder = tf.placeholder(tf.float32, shape=(batch_size, None, 1))


#
# # NTN cell implements RNNCell interface
# #ntm = tf.keras.layers.RNN(cell=cell, return_sequences=True, return_state=False,stateful=False, unroll=True)
#
# # https://www.tensorflow.org/api_docs/python/tf/compat/v1/nn/dynamic_rnn
output_sequence, _state = tf.nn.dynamic_rnn(
    cell=cell,
    inputs=inputs_placeholder,
    time_major=False,
    dtype=tf.float32,
    initial_state=None) # initial_state)


optimizer = tf.train.AdamOptimizer(learning_rate=0.001) # Starting learning rate
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=outputs_placeholder, logits=output_sequence[:, 1:, :])
loss = tf.reduce_sum(cross_entropy)/batch_size


trainable_variables = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, trainable_variables), max_grad_norm)
train_op = optimizer.apply_gradients(zip(grads, trainable_variables))

max_seq_len_placeholder = tf.placeholder(tf.int32)



In [ ]:
## Training

convergence_on_target_task = None
convergence_on_multi_task = None
performance_on_target_task = None
performance_on_multi_task = None
generalization_from_target_task = None
generalization_from_multi_task = None

data_generator = RegexDataGenerator()
target_point = args.max_seq_len
curriculum_point = 1 if args.curriculum not in ('prediction_gain', 'none') else target_point
progress_error = 1.0
convergence_error = 0.1

if args.curriculum == 'prediction_gain':
    exp3s = Exp3S(args.max_seq_len, 0.001, 0, 0.05)


sess = tf.Session()
sess.run(initializer)

